This code extracts data from the wind_data files to calculate the mean and maximum in-flight windspeed for each video clip. These variables are then stored in the env_variables_clip2.csv file.

In [2]:
# import packages
import pandas as pd
import numpy as np
import os
import glob

os.getcwd()

'/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement'

In [3]:
# set folders
save_folder = "/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement/processed-data/"

# read in necessary files and set variables
wind_files = glob.glob('/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement/clean-data/wind-data/' + '*.csv')
env_df = pd.read_csv('/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement/processed-data/env_variables_clip.csv')
clip_df = pd.read_csv('/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement/processed-data/drone_variables_clip2.csv')

In [4]:
# Get mean and max in-flight windspeed from clip ranges
env_df['max_windspeed'] = np.nan
env_df['mean_windspeed'] = np.nan
for i in wind_files:
    wind_log = pd.read_csv(i)
    flight_name = i.split('ob')[1].split('_')[0]
    if len(clip_df.loc[(clip_df['flight'] == flight_name) & (clip_df['clip_type'] == 'first')]) > 0:
        clip1_start = clip_df.loc[(clip_df['flight'] == flight_name) & (clip_df['clip_type'] == 'first'), 'clip_start_log'].values[0]
        clip1_end = clip1_start + 120000
        clip1_mean = wind_log.loc[(wind_log['milliseconds'] >= clip1_start) & (wind_log['milliseconds'] <= clip1_end), 'windspeed'].mean()
        clip1_max = wind_log.loc[(wind_log['milliseconds'] >= clip1_start) & (wind_log['milliseconds'] <= clip1_end), 'windspeed'].max()
        env_df.loc[(env_df['flight'] == flight_name) & (env_df['clip_type'] == 'first'), 'mean_windspeed'] = clip1_mean
        env_df.loc[(env_df['flight'] == flight_name) & (env_df['clip_type'] == 'first'), 'max_windspeed'] = clip1_max
    if len(clip_df.loc[(clip_df['flight'] == flight_name) & (clip_df['clip_type'] == 'second')]) > 0:
        clip2_start = clip_df.loc[(clip_df['flight'] == flight_name) & (clip_df['clip_type'] == 'second'), 'clip_start_log'].values[0]
        clip2_end = clip2_start + 120000
        clip2_mean = wind_log.loc[(wind_log['milliseconds'] >= clip2_start) & (wind_log['milliseconds'] <= clip2_end), 'windspeed'].mean()
        clip2_max = wind_log.loc[(wind_log['milliseconds'] >= clip2_start) & (wind_log['milliseconds'] <= clip2_end), 'windspeed'].max()
        env_df.loc[(env_df['flight'] == flight_name) & (env_df['clip_type'] == 'second'), 'mean_windspeed'] = clip2_mean
        env_df.loc[(env_df['flight'] == flight_name) & (env_df['clip_type'] == 'second'), 'max_windspeed'] = clip2_max

In [5]:
# Save new env_df dataframe to env_variables_clip2.csv
cols_to_keep = ['flight', 'clip_type', 'bushiness_100m',
       'max_windspeed', 'mean_windspeed']
env_df.drop(env_df.columns.difference(cols_to_keep), axis = 1, inplace = True)
env_df.to_csv('/Users/blaircostelloe/Dropbox/Publications/DroneDisturbance/drone-disturbance/supplement/processed-data/env_variables_clip2.csv', index = False)